In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
import pandas as pd
path = '/content/drive/MyDrive/머신러닝/data/labeledTrainData.tsv'
df_review = pd.read_csv(path, sep='\t', quoting=3)
df_review.head(2)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."


In [13]:
df_review['review'][0]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

In [14]:
import re
# 데이터 전처리
df_review["review"] = df_review["review"].apply( lambda x : x.replace('<br />', ' ') )
df_review["review"] = df_review["review"].apply( lambda x : re.sub("[^a-zA-Z]", " ", x) )
df_review['review'][0]

' With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay   Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him   The actual feature film bit when it finally starts is only on for 

In [18]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [19]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

senti_analyzer = SentimentIntensityAnalyzer()
senti_analyzer.polarity_scores(df_review['review'][0])

{'compound': -0.7943, 'neg': 0.13, 'neu': 0.743, 'pos': 0.127}

In [21]:
def get_vader(review):
  senti_analyzer = SentimentIntensityAnalyzer()
  scores = senti_analyzer.polarity_scores(review)
  if scores['compound']>0:
    return 1
  return 0

In [22]:
get_vader(df_review['review'][0])

0

In [23]:
df_review['pred'] = df_review['review'].apply(lambda x : get_vader(x))

In [24]:
from sklearn.datasets import fetch_20newsgroups

cats = ['rec.motorcycles', 'rec.sport.baseball', 'comp.graphics', 'comp.windows.x',
        'talk.politics.mideast', 'soc.religion.christian', 'sci.electronics', 'sci.med'  ]

df_news = fetch_20newsgroups(subset='all', remove=('header', 'footers', 'quotes'), categories=cats, random_state=62)

In [47]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(max_df=0.95, max_features=1000, min_df=2, ngram_range=(1,2), stop_words='english')
vect_data = count_vect.fit_transform(df_news.data)
vect_data.shape

(7862, 1000)

In [48]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=8, random_state=62)
lda.fit(vect_data)

LatentDirichletAllocation(n_components=8, random_state=62)

In [49]:
for idx, topic in enumerate(lda.components_):
  topic_sort_idx = topic.argsort()[::-1]
  print(topic_sort_idx[:10])
  # print(idx, ' : ', topic)
  break

[104 105 662 765 901 466 381 900 593 103]


In [51]:
feature_names = count_vect.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [54]:
for idx, topic in enumerate(lda.components_):
  topic_sort_idx = topic.argsort()[::-1]
  # print(topic_sort_idx[:10])
  # print(idx, ' : ', topic)
  # for i in topic_sort_idx[:10]:
  #   print(feature_names[i])
  print('/'.join(feature_names[i] for i in topic_sort_idx[:10]))
  # break

armenian/armenians/people/said/turkish/jews/government/turkey/muslim/armenia
image/graphics/jpeg/dos/software/file/data/images/color/edu
edu/organization/posting/university/host/nntp/nntp posting/posting host/organization university/world
god/people/think/don/know/just/say/like/organization/does
use/com/organization/window/like/problem/using/used/don/need
israel/israeli/10/1993/medical/year/health/research/00/years
organization/com/edu/posting/nntp/nntp posting/posting host/host/just/don
sun/file/mit/edu/com/mit edu/program/list/available/motif
